# Use Case: Assessment of trends in wildfire events in the Iberian Peninsula

## Import packages

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import regionmask
import scipy.stats
import shapely
import xarray as xr
from c3s_eqc_automatic_quality_control import download, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
year_start = 2001
year_stop = 2019

# Region of interest
lon_slice = slice(-10, 4)
lat_slice = slice(45, 35)

# Shapefile with regions
shapefile_url = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/shp/NUTS_RG_20M_2021_4326.shp.zip"

## Define request

In [ ]:
request = (
    "satellite-fire-burned-area",
    {
        "format": "zip",
        "origin": "esa_cci",
        "sensor": "modis",
        "version": "5_1_1cds",
        "year": [str(year) for year in range(year_start, year_stop + 1)],
        "variable": "grid_variables",
        "month": ["06", "07", "08"],
        "nominal_day": "01",
    },
)

## Download and regionalise

In [ ]:
# Download and regionalize
ds = download.download_and_transform(
    *request,
    transform_func=utils.regionalise,
    transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
    chunks={"year": 1},
)

# Reindex using year/month (shift months + 1)
ds = ds.assign_coords(
    year=("time", ds["time"].dt.year.astype(int).values),
    month=("time", ds["time"].dt.month.astype(int).values + 1),
)
ds = ds.set_index(time=("year", "month")).unstack("time")

# Convert units
da = ds["burned_area"]
with xr.set_options(keep_attrs=True):
    da *= 1.0e-6
da.attrs["units"] = "km$^2$"
da.attrs["long_name"] = "Fire Burned Area"

## Mask regions

In [ ]:
lon_bounds = [-10.18, 1]
lat_bounds = [36.5, 43]
crs = "epsg:4326"

lon_bounds += sorted(lon_bounds, reverse=True)
lat_bounds = [lat for lat in lat_bounds for _ in range(2)]
bbox = shapely.Polygon(zip(lon_bounds, lat_bounds))

gdf = gpd.read_file(shapefile_url)
gdf = gdf[gdf["LEVL_CODE"] == 2]
gdf = gdf[gdf.intersects(bbox)]
gdf = gdf[gdf["NUTS_ID"].str.startswith(("ES", "PT"))]
gdf = gdf.to_crs(crs)

ds.rio.write_crs(crs, inplace=True)

regions = regionmask.from_geopandas(gdf, names="NUTS_NAME")
mask = regions.mask_3D(ds["longitude"], ds["latitude"])

## Compute and add stats

In [ ]:
da_grouped = da.where(mask).sum(("latitude", "longitude", "month"))
da_grouped = da_grouped.assign_coords(region=mask["names"]).compute()

df_dict = {"slope": [], "intercept": [], "tau": [], "p-value": []}
for region in gdf["NUTS_NAME"]:
    da_region = da_grouped.sel(region=region).squeeze()
    slope, intercept, *_ = scipy.stats.theilslopes(da_region, da_region["year"])
    tau, p_value = scipy.stats.kendalltau(da_region["year"], da_region)
    df_dict["slope"].append(slope)
    df_dict["intercept"].append(intercept)
    df_dict["tau"].append(tau)
    df_dict["p-value"].append(p_value)
gdf[list(df_dict)] = pd.DataFrame(df_dict, index=gdf.index)

## Define plotting function

In [ ]:
def plot_trend(da, df, **kwargs):
    facet = da.plot(col="region", marker="o", sharex=False, **kwargs)
    for ax, sel_dict in zip(facet.axs.flatten(), facet.name_dicts.flatten()):
        if not sel_dict:
            continue
        da_sel = da.sel(sel_dict)
        df_region = df[df["NUTS_NAME"] == sel_dict["region"]].squeeze()
        ax.plot(
            da_sel["year"],
            da_sel["year"] * df_region["slope"] + df_region["intercept"],
            label="\n".join(
                [
                    f"slope = {df_region['slope']:.3f} km$^2$/year",
                    f"p-value = {df_region['p-value']:.3f}",
                    f"tau = {df_region['tau']:.3f}",
                    f"intercept = {df_region['intercept']:.3f} km$^2$",
                ]
            ),
            color="k",
            ls="--",
        )
        ax.set_xticks(da_sel["year"][::2])
        ax.set_title(sel_dict["region"])
        ax.legend()
        ax.grid()
    return facet

## Plot all regions - timeseries

In [ ]:
facet = plot_trend(da_grouped, gdf, col_wrap=3, aspect=1.5)

## Plot regions of interest - timeseries

In [ ]:
regions_of_interest = ["Galicia", "Andalucía", "Norte", "Centro (PT)"]
_ = plot_trend(da_grouped.sel(region=regions_of_interest), gdf, col_wrap=1, aspect=2)

## Plot trends map

In [ ]:
vmax = gdf["slope"].abs().max()
ax = gdf.plot(
    gdf["slope"],
    cmap="coolwarm",
    vmax=vmax,
    vmin=-vmax,
    edgecolor="k",
    legend=True,
    legend_kwds={"label": "Trend Value [km$^2$/year]"},
)
ax = gdf.loc[~(gdf["p-value"] < 0.05)].plot(facecolor="none", hatch="///", ax=ax)
_ = ax.axis("off")